In [12]:
!pip install diaparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 10.6 MB/s eta 0:00:00


In [13]:
texts = []

with open('/kaggle/input/mahabharat-index/cleanedTextFileFinalDONE.txt', 'r',encoding='utf-8') as file:
    for line in file:
        texts.append(line[:-1])
        


In [14]:
texts[20:30]

['Abhīru, a king, who was an incarnation of the sixth of the eight Asuras of the family of the Kāleyāḥ, .',
 'Abhisāra, a people  . Together with other barbarous tribes they fight Arjuna on the th day of the great battle, . They are mentioned by Kṛṣṇa among the auxiliaries of Duryodhana, on the th day of the battle.',
 'Abhīsāra, enumerated by Sañjaya among the people of Bhāratavarṣa, .',
 'Abhisārī, a town conquered by Arjuna  on his Digvijaya, .',
 'Abhiṣāha, a people  with other barbarous tribes they attack Arjuna at the instigation of Duḥśāsana on the th day of the great battle, . .',
 'Abhīṣāha, a people   first day of the battle, .  th day, .  th day, .  th day, . After the th day, .',
 'Ābhiṣecanika parva, i.e .',
 'Abhiṣyanta, the second of the five sons of Kuru and Vāhinī, .',
 'Abhivādya = Śiva',
 'Acala1, brother in law of Dhṛtāraṣṭra, brother of Śakuni, .']

In [15]:
def extract_rel_window(text_list, keywords_list):
    sents = []
    for sent in text_list: 
        for keyword in keywords_list:
            if keyword in sent:
                index = sent.find(keyword)
                min_idx = index
                max_idx = index
                
                while min_idx>=0 and sent[min_idx]!='.':
                    min_idx = min_idx-1
                while max_idx<len(sent) and sent[max_idx]!='.':
                    max_idx = max_idx+1
                
                # max_idx = len(sent)-1
                # if index+window_size<len(sent)-1:
                #     max_idx = index+window_size
                min_idx = min_idx + 1
                sents.append(sent[min_idx:max_idx])
    return sents

In [16]:
sents = extract_rel_window(texts,["father of"])
sents = [sent.lower() for sent in sents]
sents

['aśvapati2, a king of the madras, father of sāvitrī, ',
 'adhiratha, a sūta in the neighbourhood of campā and friend of dhṛtarāṣṭra husband of rādhā and foster father of karṇa, whom he called vasuṣeṇa and sent to hāstinapura to be instructed by droṇa  at the show of arms he enters the arena to congratulate karṇa as king of aṅga     ',
 'ahaṃyāti, a king, son of saṃyāti and father of sārvabhauma ',
 'anala1, one of the eight vasus, son of prajāpati  or dharma  and śāṇḍilī, and father of skanda',
 ' hiraṇyakaśipu, who was the father of mandāra, irresistible even to viṣṇus discus sudarśana  śiva had further granted boons to the dānavas  vidyutprabha, śatamukha, and to brahmán, etc',
 'aṣṭaka   yayāti, who was the maternal grandfather of a discoursed with him on haughtiness, etc and told him his own history yayāti spoke to a',
 'aurva   son of cyavana and āruṣī and father of ṛcīka, ',
 'āyunāthānaraṇyenakṛṣṇason of purūravas and father of nahuṣaānuśāsan',
 ' b, kṛṣṇa vāsudevamarried to il

## Removing trailing commas and putting a full stop

In [17]:
def remove_before_last_alphabet(sentence):
    # Find the index of the last alphabet
    last_alpha_index = None
    for i in range(len(sentence) - 1, -1, -1):
        if sentence[i].isalpha():
            last_alpha_index = i
            break
    
    # Remove everything before the last alphabet
    if last_alpha_index is not None:
        sentence = sentence[0:last_alpha_index+1]
    
    # Add a full stop at the end
    sentence = sentence.strip() + "."
    
    return sentence

def assign_token_ids(sent):
    # Initialize an empty list to store token information
    tokens = []

    # Iterate over each token in the sentence
    for i, (word, head, rel) in enumerate(zip(sent.words, sent.arcs, sent.rels), start=1):
        # Extract relevant information from each token
        token_info = {
            "id": i,
            "word": word,
            "head": head,
            "rel": rel,
        }
        tokens.append(token_info)

    return tokens

import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet

def is_english_word(word):
    # Check if the word is in WordNet's list of English words
    return wordnet.synsets(word) != []

def transliterate_to_latin(word):
    iast_mapping = {
        'á': 'aa',
        'ñ': 'n',
        'ā': 'aa',
        'ă': 'aa',
        'ī': 'i',
        'ĭ': 'i',
        'Ś': 'sh',
        'ś': 'sh',
        'Ū': 'u',
        'ū': 'u',
        'ḍ': 'd',
        'Ḣ': '',
        'ḥ': '',
        'ṃ': 'm',
        'ṅ': 'n',
        'ṇ': 'n',
        'ṛ': 'ri',
        'ṝ': 'ri',
        'ṣ': 'sh',
        'ṭ': 't'
    }

    latin_word = ''
    for char in word:
        if char in iast_mapping:
            latin_word += iast_mapping[char]
        else:
            latin_word += char
    return latin_word





[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Dependency parsing trials - **FATHER**

In [18]:
from diaparser.parsers import Parser

In [19]:
parser = Parser.load('en_ewt.electra-base')

Downloading: "https://github.com/Unipisa/diaparser/releases/download/v1.0/en_ewt.electra-base" to /root/.cache/diaparser/en_ewt.electra-base
100%|██████████| 452M/452M [00:06<00:00, 72.5MB/s] 


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [20]:
text = "Aśvapati2, a king of the Madras, father of Sāvitrī,"
dataset = parser.predict(text, text='en')
dataset.sentences[0]

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:28: UserWarning: apply_permutation is deprecated, please use tensor.index_select(dim, permutation) instead
  warnings.warn("apply_permutation is deprecated, please use tensor.index_select(dim, permutation) instead")


# sent_id = 1
# text = Aśvapati2, a king of the Madras, father of Sāvitrī,
1	Aśvapati2	_	_	_	_	0	root	_	_
2	,	_	_	_	_	1	punct	_	_
3	a	_	_	_	_	4	det	_	_
4	king	_	_	_	_	1	appos	_	_
5	of	_	_	_	_	7	case	_	_
6	the	_	_	_	_	7	det	_	_
7	Madras	_	_	_	_	4	nmod	_	_
8	,	_	_	_	_	4	punct	_	_
9	father	_	_	_	_	4	conj	_	_
10	of	_	_	_	_	11	case	_	_
11	Sāvitrī	_	_	_	_	9	nmod	_	_
12	,	_	_	_	_	1	punct	_	_

In [21]:
text = "Aśvapati2, a king of the Madras, father of Sāvitrī."
dataset = parser.predict(text, text='en')
dataset.sentences[0]

# sent_id = 1
# text = Aśvapati2, a king of the Madras, father of Sāvitrī.
1	Aśvapati2	_	_	_	_	0	root	_	_
2	,	_	_	_	_	1	punct	_	_
3	a	_	_	_	_	4	det	_	_
4	king	_	_	_	_	1	appos	_	_
5	of	_	_	_	_	7	case	_	_
6	the	_	_	_	_	7	det	_	_
7	Madras	_	_	_	_	4	nmod	_	_
8	,	_	_	_	_	4	punct	_	_
9	father	_	_	_	_	1	appos	_	_
10	of	_	_	_	_	11	case	_	_
11	Sāvitrī	_	_	_	_	9	nmod	_	_
12	.	_	_	_	_	1	punct	_	_

In [22]:
text = "Ahaṃyāti, a king, son of Saṃyāti and father of Sārvabhauma."
dataset = parser.predict(text, text='en')
dataset.sentences[0]

# sent_id = 1
# text = Ahaṃyāti, a king, son of Saṃyāti and father of Sārvabhauma.
1	Ahaṃyāti	_	_	_	_	0	root	_	_
2	,	_	_	_	_	1	punct	_	_
3	a	_	_	_	_	4	det	_	_
4	king	_	_	_	_	1	appos	_	_
5	,	_	_	_	_	1	punct	_	_
6	son	_	_	_	_	1	appos	_	_
7	of	_	_	_	_	8	case	_	_
8	Saṃyāti	_	_	_	_	6	nmod	_	_
9	and	_	_	_	_	10	cc	_	_
10	father	_	_	_	_	8	conj	_	_
11	of	_	_	_	_	12	case	_	_
12	Sārvabhauma	_	_	_	_	10	nmod	_	_
13	.	_	_	_	_	4	punct	_	_

In [23]:
text = "Aurva son of Cyavana and Āruṣī and father of Ṛcīka, "
dataset = parser.predict(text, text='en')
dataset.sentences[0]

# sent_id = 1
# text = Aurva son of Cyavana and Āruṣī and father of Ṛcīka,
1	Aurva	_	_	_	_	0	root	_	_
2	son	_	_	_	_	1	appos	_	_
3	of	_	_	_	_	4	case	_	_
4	Cyavana	_	_	_	_	2	nmod	_	_
5	and	_	_	_	_	6	cc	_	_
6	Āruṣī	_	_	_	_	4	conj	_	_
7	and	_	_	_	_	8	cc	_	_
8	father	_	_	_	_	1	conj	_	_
9	of	_	_	_	_	10	case	_	_
10	Ṛcīka	_	_	_	_	8	nmod	_	_
11	,	_	_	_	_	1	punct	_	_

In [24]:
text = "Cikura, a serpent, father of the serpent Sumukha ."
dataset = parser.predict(text, text='en')
dataset.sentences[0]

# sent_id = 1
# text = Cikura, a serpent, father of the serpent Sumukha .
1	Cikura	_	_	_	_	0	root	_	_
2	,	_	_	_	_	1	punct	_	_
3	a	_	_	_	_	4	det	_	_
4	serpent	_	_	_	_	1	appos	_	_
5	,	_	_	_	_	1	punct	_	_
6	father	_	_	_	_	1	appos	_	_
7	of	_	_	_	_	9	case	_	_
8	the	_	_	_	_	9	det	_	_
9	serpent	_	_	_	_	6	nmod	_	_
10	Sumukha	_	_	_	_	9	appos	_	_
11	.	_	_	_	_	1	punct	_	_

In [25]:
text = "Hiraṇyakaśipu, who was the father of mandāra, irresistible even to viṣṇus discus sudarśana śiva had further granted boons to the dānavas vidyutprabha, śatamukha, and to brahmán, etc. "
dataset = parser.predict(text, text='en')
dataset.sentences[0]

# sent_id = 1
# text = Hiraṇyakaśipu, who was the father of mandāra, irresistible even to viṣṇus discus sudarśana śiva had further granted boons to the dānavas vidyutprabha, śatamukha, and to brahmán, etc.
1	Hiraṇyakaśipu	_	_	_	_	19	nsubj	_	_
2	,	_	_	_	_	1	punct	_	_
3	who	_	_	_	_	6	nsubj	_	_
4	was	_	_	_	_	6	cop	_	_
5	the	_	_	_	_	6	det	_	_
6	father	_	_	_	_	1	acl:relcl	_	_
7	of	_	_	_	_	8	case	_	_
8	mandāra	_	_	_	_	6	nmod	_	_
9	,	_	_	_	_	10	punct	_	_
10	irresistible	_	_	_	_	6	amod	_	_
11	even	_	_	_	_	14	advmod	_	_
12	to	_	_	_	_	14	case	_	_
13	viṣṇus	_	_	_	_	14	compound	_	_
14	discus	_	_	_	_	10	obl	_	_
15	sudarśana	_	_	_	_	19	nsubj	_	_
16	śiva	_	_	_	_	15	flat	_	_
17	had	_	_	_	_	19	aux	_	_
18	further	_	_	_	_	19	advmod	_	_
19	granted	_	_	_	_	0	root	_	_
20	boons	_	_	_	_	19	obj	_	_
21	to	_	_	_	_	24	case	_	_
22	the	_	_	_	_	24	det	_	_
23	dānavas	_	_	_	_	24	compound	_	_
24	vidyutprabha	_	_	_	_	19	obl	_	_
25	,	_	_	_	_	26	punct	_	_
26	śatamukha	_	_	_	_	24	conj	_	_
27	,	_	_	_	_	30	punct	_	_
28	and	_

In [26]:
text = "śūra2, a yādava, father of vasudeva and kuntī"
dataset = parser.predict(text, text='en')
dataset.sentences[0]

# sent_id = 1
# text = śūra2, a yādava, father of vasudeva and kuntī
1	śūra2	_	_	_	_	0	root	_	_
2	,	_	_	_	_	1	punct	_	_
3	a	_	_	_	_	4	det	_	_
4	yādava	_	_	_	_	1	appos	_	_
5	,	_	_	_	_	1	punct	_	_
6	father	_	_	_	_	4	appos	_	_
7	of	_	_	_	_	8	case	_	_
8	vasudeva	_	_	_	_	6	nmod	_	_
9	and	_	_	_	_	10	cc	_	_
10	kuntī	_	_	_	_	8	conj	_	_

# Function to extract relations

In [30]:
#TEMP
def extract_rel(sents,rel):
    ans_dict = []
    my_dict = {}
    found_relations=[]
    
    
    # sentences_10 = [ "He explained to him why kine have descended to earth In days of yore, when in the Deva yuga the Dānavas became lords of the three worlds, Aditi underwent the severest austerities and begot Viṣṇu seeing Aditi perform penances, Surabhī, the daughter of Dakṣa, also underwent austerities on Kailāsa, in yoga, standing upon one leg for, years Ṛ and M."]
    
    for sent in sents:
        if len(sent)<5:
            continue
        sent = remove_before_last_alphabet(sent)
        sent = transliterate_to_latin(sent)
        a = assign_token_ids((parser.predict(sent, text='en')).sentences[0])
        # finding head 
        head=""
        id=0
        for i in a:
    #         print(i)
            if i['word']==rel:
                id  = i['id']
                head_pointer = i['head']
                for j in a:
                    if j['id']==head_pointer:
                        head = j['word']
    
        if head=="":
            continue
            
        # finding tail 
        tails = []
        tail = ""
        for i in range(id+1 , len(a)):
            if a[i]['rel'] != 'nmod' or a[i]['word'] == 'king' or a[i]['word'] == 'queen':
                tail = tail + a[i]['word'] + " "
            else:
                tails.append(tail+a[i]['word'])
                j=i+1
                #find multiple tails if possible:
                while(j<len(a) and (a[j]['word'] in ['and',','] or a[j]['rel'] == 'conj')):
                    if a[j]['word']=='and' or a[j]['word']==',':
                        del a[j]
                    elif a[j]['rel'] == 'conj':
                        tails.append(a[j]['word'])
                        j=j+1
                    
                break

        tails = [tail for tail in tails if len(tail.split()) < 6]
        
        # print(head , 'father ', tail , 'text : ', sent,'\n')
        # found_relations.append(head + ' father ' + tail )

        for tail in tails:
            my_dict = {}
            my_dict['head']=head
            my_dict['relation']=rel+' of'
            my_dict['tail']=tail
            my_dict['text']=sent
            ans_dict.append(my_dict)
            
    
    return ans_dict    
    
    
# printing answers

# for i in found_relations:
#     print(i)


    
    
# print(found_relations)
# print(ans_dict)

In [31]:
len(sents)

91

In [32]:
rels = extract_rel(sents,"father")

In [36]:
rels

[{'head': 'king',
  'relation': 'father of',
  'tail': 'saavitri',
  'text': 'ashvapati2, a king of the madras, father of saavitri.'},
 {'head': 'suta',
  'relation': 'father of',
  'tail': 'karna',
  'text': 'adhiratha, a suta in the neighbourhood of campaa and friend of dhritaraashtra husband of raadhaa and foster father of karna, whom he called vasushena and sent to haastinapura to be instructed by drona  at the show of arms he enters the arena to congratulate karna as king of anga.'},
 {'head': 'samyaati',
  'relation': 'father of',
  'tail': 'saarvabhauma',
  'text': 'ahamyaati, a king, son of samyaati and father of saarvabhauma.'},
 {'head': 'prajaapati',
  'relation': 'father of',
  'tail': 'skanda',
  'text': 'anala1, one of the eight vasus, son of prajaapati  or dharma  and shaandili, and father of skanda.'},
 {'head': 'hiranyakashipu',
  'relation': 'father of',
  'tail': 'mandaara',
  'text': 'hiranyakashipu, who was the father of mandaara, irresistible even to vishnus discu

In [37]:
import pickle

with open('FATHER.pkl', 'wb') as f:
    pickle.dump(rels, f)

In [255]:
text = "kaashya1, the father of ambaa, ambikaa, and ambaalikaa."
dataset = parser.predict(text, text='en')
dataset.sentences[0]

# sent_id = 1
# text = kaashya1, the father of ambaa, ambikaa, and ambaalikaa.
1	kaashya1	_	_	_	_	0	root	_	_
2	,	_	_	_	_	1	punct	_	_
3	the	_	_	_	_	4	det	_	_
4	father	_	_	_	_	1	appos	_	_
5	of	_	_	_	_	6	case	_	_
6	ambaa	_	_	_	_	4	nmod	_	_
7	,	_	_	_	_	8	punct	_	_
8	ambikaa	_	_	_	_	6	conj	_	_
9	,	_	_	_	_	11	punct	_	_
10	and	_	_	_	_	11	cc	_	_
11	ambaalikaa	_	_	_	_	6	conj	_	_
12	.	_	_	_	_	1	punct	_	_

In [273]:
extract_rel(["kaashya1, the father of ambaa, ambikaa, ruby,and anshul and vipul"], "father")

['ambaa', 'ambikaa', 'ruby', 'anshul', 'vipul']


[{'head': 'kaashya1',
  'relation': 'father of',
  'tail': 'ambaa',
  'text': 'kaashya1, the father of ambaa, ambikaa, ruby,and anshul and vipul.'},
 {'head': 'kaashya1',
  'relation': 'father of',
  'tail': 'ambikaa',
  'text': 'kaashya1, the father of ambaa, ambikaa, ruby,and anshul and vipul.'},
 {'head': 'kaashya1',
  'relation': 'father of',
  'tail': 'ruby',
  'text': 'kaashya1, the father of ambaa, ambikaa, ruby,and anshul and vipul.'},
 {'head': 'kaashya1',
  'relation': 'father of',
  'tail': 'anshul',
  'text': 'kaashya1, the father of ambaa, ambikaa, ruby,and anshul and vipul.'},
 {'head': 'kaashya1',
  'relation': 'father of',
  'tail': 'vipul',
  'text': 'kaashya1, the father of ambaa, ambikaa, ruby,and anshul and vipul.'}]

In [278]:
extract_rel(["raibhya, a brahman, father of arvāvasu and parāvasu"],'father')

['arvaavasu', 'paraavasu']


[{'head': 'raibhya',
  'relation': 'father of',
  'tail': 'arvaavasu',
  'text': 'raibhya, a brahman, father of arvaavasu and paraavasu.'},
 {'head': 'raibhya',
  'relation': 'father of',
  'tail': 'paraavasu',
  'text': 'raibhya, a brahman, father of arvaavasu and paraavasu.'}]

In [275]:
extract_rel(["parikṣit or parīkṣit, son of abhimanyu and uttarā, and father of janamejaya."],'father')

['janamejaya']


[{'head': 'parikshit',
  'relation': 'father of',
  'tail': 'janamejaya',
  'text': 'parikshit or parikshit, son of abhimanyu and uttaraa, and father of janamejaya.'}]